In [145]:
# 필요한 라이브러리 로드
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn import preprocessing as pp
from keras.models import Sequential
from sklearn.metrics import accuracy_score

In [146]:
df= pd.read_csv('/content/drive/MyDrive/WISDM.csv').drop('Unnamed: 0', axis = 1)


Jogging = df[df['activity']=='Jogging'].head(4500).copy()
Sitting = df[df['activity']=='Sitting'].head(4500).copy()

balanced_data = pd.DataFrame()

balanced_data = pd.concat([balanced_data, Jogging,  Sitting])

balanced_data.shape


(9000, 6)

In [147]:
datax = balanced_data.drop(['activity', 'timestamp' , 'user-id'  ], axis =1)
datay = balanced_data['activity']

In [148]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(balanced_data.activity)
datay = encoder.transform(datay)


In [149]:
featuresToScale = datax.columns

sX = pp.StandardScaler(copy=True, with_mean=True, with_std=True)
datax.loc[:,featuresToScale] = sX.fit_transform(datax[featuresToScale])

In [150]:
X_train, X_test, y_train, y_test = \
    train_test_split(datax, datay, test_size=0.33, \
                     random_state=2018, stratify=datay)

In [151]:
X_train_AE = X_train.copy()
X_test_AE = X_test.copy()


In [152]:
input_dim = X_train.shape[1]
encoding_dim = 14

input_layer = Input(shape=(input_dim, ))

encoder = Dense(encoding_dim, activation="tanh",
                activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim / 2), activation="relu")(encoder)
decoder = Dense(int(encoding_dim / 2), activation='tanh')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

In [153]:
test_scores = []

noise_factor = 0.50
X_train_AE_noisy = X_train_AE.copy() + noise_factor * \
 np.random.normal(loc=0.0, scale=1.0, size=X_train_AE.shape)
X_test_AE_noisy = X_test_AE.copy() + noise_factor * \
 np.random.normal(loc=0.0, scale=1.0, size=X_test_AE.shape)

#nb_epoch = 100
nb_epoch = 10
batch_size = 32
autoencoder.compile(optimizer='adam',
                    loss='mean_squared_error',
                    metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath="model.h5",
                               verbose=0,
                               save_best_only=True)
tensorboard = TensorBoard(log_dir='./logs',
                          histogram_freq=0,
                          write_graph=True,
                          write_images=True)
history = autoencoder.fit(X_train_AE_noisy, X_train_AE_noisy,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_test_AE_noisy, X_test_AE_noisy),
                    verbose=1,
                    callbacks=[checkpointer, tensorboard]).history


Epoch 1/10
189/189 [==============================] - 2s 4ms/step - loss: 0.8392 - accuracy: 0.7556 - val_loss: 0.7176 - val_accuracy: 0.9320
Epoch 2/10
 72/189 [==========>...................] - ETA: 0s - loss: 0.6717 - accuracy: 0.9427

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


189/189 [==============================] - 1s 3ms/step - loss: 0.6592 - accuracy: 0.9479 - val_loss: 0.6825 - val_accuracy: 0.9485
Epoch 3/10
189/189 [==============================] - 0s 2ms/step - loss: 0.6379 - accuracy: 0.9434 - val_loss: 0.6688 - val_accuracy: 0.9414
Epoch 4/10
189/189 [==============================] - 0s 2ms/step - loss: 0.6303 - accuracy: 0.9386 - val_loss: 0.6648 - val_accuracy: 0.9377
Epoch 5/10
189/189 [==============================] - 0s 2ms/step - loss: 0.6275 - accuracy: 0.9353 - val_loss: 0.6626 - val_accuracy: 0.9357
Epoch 6/10
189/189 [==============================] - 0s 2ms/step - loss: 0.6255 - accuracy: 0.9345 - val_loss: 0.6608 - val_accuracy: 0.9357
Epoch 7/10
189/189 [==============================] - 0s 2ms/step - loss: 0.6236 - accuracy: 0.9345 - val_loss: 0.6592 - val_accuracy: 0.9384
Epoch 8/10
189/189 [==============================] - 0s 2ms/step - loss: 0.6221 - accuracy: 0.9365 - val_loss: 0.6579 - val_accuracy: 0.9424
Epoch 9/10
189/18

In [154]:
threshold = 0.4
predictions = autoencoder.predict(X_test_AE_noisy)
mse = np.mean(np.power(X_test_AE_noisy - predictions, 2), axis=1)
error_df = pd.DataFrame({'reconstruction_error': mse,
                        'true_class': y_test})
error_df.describe()

93/93 [==============================] - 0s 915us/step


,reconstruction_error,true_class
count,2970.000000,2970.000000
mean,0.655418,0.500000
std,0.961155,0.500084
min,0.000058,0.000000
25%,0.056918,0.000000
50%,0.349547,0.500000
75%,0.924202,1.000000
max,11.483251,1.000000


In [155]:
y_pred = [0 if e > threshold else 1 for e in error_df.reconstruction_error.values]
precision, recall, f1,_ = precision_recall_fscore_support(y_test,y_pred,average='binary')

print ('Accuracy Score :',accuracy_score(error_df.true_class, y_pred) )
print ('Precision :',precision )
print ('Recall :',recall )
print ('F1 :',f1 )



Accuracy Score : 0.4686868686868687
Precision : 0.4704761904761905
Recall : 0.498989898989899
F1 : 0.4843137254901961
